#### API Alpha_vantage is an API to download information on different asset classes
#### It has its own python library

In [3]:
from bs4 import BeautifulSoup as bs
import requests
import re
import json
import pandas as pd
import os
import glob
from functools import reduce
from alpha_vantage.timeseries import TimeSeries

#### Download the share prices per company in the share list and save them into csv files

In [40]:
with open("alpha_vantage_access.json", "r") as infile:
    my_access_key = json.loads(infile.read())

#   This app allows max 5 calls per mins, thereofere I have to break it down to 6 times in order to get the 30 files 
df_symbol = pd.read_csv("Stocklist_DAX")
symbol_list = list(df_symbol["Symbol"][0:5])

start_date = pd.to_datetime("2020-01-01")
end_date = pd.to_datetime("2020-09-30")

for symbol in symbol_list:
    app = TimeSeries(key="my_access_key", output_format="pandas")
    tuple_shares = app.get_daily_adjusted(symbol, outputsize='full')
    df_share = tuple_shares[0]
    df = df_share.reset_index()
    mask = (df['date'] >= start_date ) & (df['date'] <= end_date)
    df_2020 = df[mask]
    filename = "\\DAX_Share\\DAX_Share_Prices\\df_" + symbol
    df_2020.to_csv(os.getcwd() + filename) 

In [ ]:
'''Some useful check
df_addidas.head(2)
df_addidas.iloc[0,0]
df.shape
df.head()
df.iloc[0,0]
df_addidas.info()'''

#### The plan is to have all the information about all the 30 shares, but unfortunately the API did not recognised several stock ticker and when we checked the downloaded files, there are files with very small size such as 1KB which cannot be correct. We have to exclude those shares due to no value. We decided to find DAX through API and we found out the issue is due to the API is probably get financial information from US stock exchange, the stock symbol in different stock exchange has slight difference, that could be the reason why we cannot get all the share prices as planned. We decided to include another column which is a DAX ETF listed in US.

In [ ]:
'''with open("alpha_vantage_access.json", "r") as infile:
    my_access_key = json.loads(infile.read())

start_date = pd.to_datetime("2020-01-01")
end_date = pd.to_datetime("2020-09-30")

app = TimeSeries(key="my_access_key", output_format="pandas")
tuple_shares = app.get_daily_adjusted("DAX", outputsize='full')
df_share = tuple_shares[0]
df = df_share.reset_index()
mask = (df['date'] >= start_date ) & (df['date'] <= end_date)
df_2020 = df[mask]
filename = "\\DAX_Share\\DAX_Share_Prices\\df_" + "DAX"
df_2020.to_csv(os.getcwd() + filename)'''

#### Merge all the individual share price file into one dataframe by date.

In [26]:
path = "DAX_Share/DAX_Share_Prices"
all_filenames = os.listdir(path)
all_filenames
#combine all files in the list
file_all = []
for file in all_filenames:
    filepath = path + "/" + file
    df = pd.read_csv(filepath)
    df = df[["date", "5. adjusted close"]]
    df_new = df.sort_values(by="date")
    df_new = df_new.rename(columns={"date":"Date", "5. adjusted close":"close_" + file})
    file_all.append(df_new)
file_all

share_merged = reduce(lambda df1,df2: pd.merge(df1,df2, how="left", on='Date'), file_all)
share_merged = share_merged.sort_values(by="Date")
share_merged

,Date,close_df_ADS,close_df_ALV,close_df_BAYN,close_df_DAX,close_df_DTE,close_df_HEI,close_df_LIN,close_df_MRK,close_df_MTX,close_df_SAP
0,2020-01-02,110.6761,85.0618,NaN,28.1798,125.0174,119.6146,207.9681,89.9381,57.2792,134.8236
1,2020-01-03,108.6098,82.6711,NaN,27.6755,125.3185,120.3036,202.5602,89.1662,56.5513,132.0347
2,2020-01-06,109.1018,81.3915,NaN,27.6903,125.2894,121.2622,201.7017,89.5473,55.8632,132.2515
3,2020-01-07,108.4819,81.3320,NaN,27.6409,124.8717,122.1808,202.1359,87.1630,55.5241,131.6799
4,2020-01-08,108.4425,81.4212,NaN,27.9425,124.8037,120.5232,204.6622,86.5767,55.9729,133.1187
...,...,...,...,...,...,...,...,...,...,...,...
184,2020-09-24,42.3200,69.3600,0.0,27.5000,112.9900,101.4800,234.5000,83.1500,48.1000,153.3400
185,2020-09-25,42.0000,69.1200,0.0,27.4500,114.7200,104.1700,232.7500,82.9300,49.2300,152.0600
186,2020-09-28,43.2600,72.0600,0.0,28.2300,114.8100,105.4400,237.0500,82.7600,51.1900,157.0800
187,2020-09-29,42.8200,72.8700,0.0,28.3006,114.4000,104.4400,236.4300,81.9000,51.1600,156.5700


#### Check if we have value on every trading day and some other statistic information of the dataset.

In [17]:
share_merged.isnull().sum()

Date               0
close_df_ADS       0
close_df_ALV       0
close_df_BAYN    135
close_df_DAX       0
close_df_DTE       0
close_df_HEI       0
close_df_LIN       0
close_df_MRK       0
close_df_MTX       0
close_df_SAP       0
dtype: int64

In [18]:
share_merged.describe()

,close_df_ADS,close_df_ALV,close_df_BAYN,close_df_DAX,close_df_DTE,close_df_HEI,close_df_LIN,close_df_MRK,close_df_MTX,close_df_SAP
count,189.000000,189.000000,54.000000,189.000000,189.000000,189.000000,189.000000,189.000000,189.000000,189.000000
mean,57.415610,66.479358,0.000457,25.576866,110.721328,101.563211,209.813781,80.022113,47.159187,135.314272
std,25.741245,9.934285,0.002398,3.217541,12.322439,16.201776,27.743402,4.576134,7.166391,19.845641
min,22.382800,40.180000,0.000000,17.189600,75.395500,61.610500,148.752500,65.399300,28.844600,92.763300
25%,42.321300,61.890000,0.000000,22.642700,102.870000,90.747100,187.626400,76.735900,43.158000,117.548100
50%,45.874400,67.250000,0.000000,26.854800,111.900000,103.836600,207.968100,80.098100,48.273000,134.823600
75%,62.921400,74.556800,0.000000,28.070000,117.607100,111.680000,238.130000,83.488800,52.149300,156.280000
max,112.201300,85.061800,0.014700,29.640000,131.098200,129.655900,260.230000,89.938100,58.745100,169.020000


#### We dropped one column named df_BAYN due to no enough values

In [19]:
share_merged = share_merged.drop("close_df_BAYN", axis=1)
share_merged

,Date,close_df_ADS,close_df_ALV,close_df_DAX,close_df_DTE,close_df_HEI,close_df_LIN,close_df_MRK,close_df_MTX,close_df_SAP
0,2020-01-02,110.6761,85.0618,28.1798,125.0174,119.6146,207.9681,89.9381,57.2792,134.8236
1,2020-01-03,108.6098,82.6711,27.6755,125.3185,120.3036,202.5602,89.1662,56.5513,132.0347
2,2020-01-06,109.1018,81.3915,27.6903,125.2894,121.2622,201.7017,89.5473,55.8632,132.2515
3,2020-01-07,108.4819,81.3320,27.6409,124.8717,122.1808,202.1359,87.1630,55.5241,131.6799
4,2020-01-08,108.4425,81.4212,27.9425,124.8037,120.5232,204.6622,86.5767,55.9729,133.1187
...,...,...,...,...,...,...,...,...,...,...
184,2020-09-24,42.3200,69.3600,27.5000,112.9900,101.4800,234.5000,83.1500,48.1000,153.3400
185,2020-09-25,42.0000,69.1200,27.4500,114.7200,104.1700,232.7500,82.9300,49.2300,152.0600
186,2020-09-28,43.2600,72.0600,28.2300,114.8100,105.4400,237.0500,82.7600,51.1900,157.0800
187,2020-09-29,42.8200,72.8700,28.3006,114.4000,104.4400,236.4300,81.9000,51.1600,156.5700
